In [1]:
#The purpose of this script is to clean the news header data by matching the keyword to the Mcdonald dictionary
#To do = extract mood of the keyword
import pandas as pd
import nltk
from itertools import chain
from nltk.corpus import stopwords

In [2]:
#Load Mcdonald'sdictionary
LMD = pd.read_excel("LoughranMcDonald_MasterDictionary_2014.xlsx")

In [3]:
#The original file use year instead of flag variable
#Change to flag variable 1 = indicate the sentiment
LMD.loc[LMD.Negative != 0 , 'Negative'] = 1
LMD.loc[LMD.Positive != 0 , 'Positive'] = 1
LMD.loc[LMD.Uncertainty != 0 , 'Uncertainty'] = 1
LMD.loc[LMD.Litigious != 0 , 'Litigious'] = 1
LMD.loc[LMD.Constraining != 0, 'Constraining']=1
LMD.loc[LMD.Superfluous != 0 , 'Superfluous'] = 1
LMD.loc[LMD.Interesting != 0 , 'Interesting'] = 1
LMD.loc[LMD.Modal != 0 , 'Interesting'] = 1
LMD.loc[LMD.Irr_Verb != 0 , 'Interesting'] = 1
LMD.loc[LMD.Harvard_IV != 0 , 'Interesting'] = 1

In [4]:
#Change from list of list to list
main_word = LMD[["Word"]].values.flatten().tolist()

In [5]:
#Remove stop words
stop_words = stopwords.words('english')

In [6]:
news_data = pd.read_json("WSJ/wsj.json")
#Convert from list of list to list using .str[0]
news_data["title"] = news_data["title"].str[0]    
news_data["category"] = news_data["category"].str[0]    
news_data["date"] = news_data["date"].str[0]    

In [10]:
#Extract date from the given string#

final_date = [] 
lookup_month = ["jan" , "feb" , "mar" , "apr" , "may" , "jun" , "jul" , "aug" , "sep" , "oct" , "nov" , "dec"]
row = news_data.shape[0] 
track = 1 #just to track progress

for i in range(row):
    text = news_data["date"][i]
    text = text.lower()
    
    #split by comma (month and day before comma and after comma is year)
    temp = text.split(",")
    
    #Get day
    day = temp[0][-2:].replace("  ", "")
    
    #Get year
    year = temp[1][0:5].replace(" ","")
    
    #Get month
    for m  in lookup_month:
        if temp[0].find(m) != -1:
            month = m 
    
    '''
    #Print process
    if i/10000 > track:
        print(i)
        track = track + 1
        
    '''
    
    final_date.append(day + "-" + month + "-" + year)

In [11]:
final_score = []
Negative_final = []
Positive_final = []
Uncertainty_final = []
Litigious_final = []
Constraining_final = []
Superfluous_final = []
Interesting_final = [] 
Modal_final = []
Irr_Verb_final = []
Harvard_IV_final = []

track = 1

for i in range(row): #For each news header
# for i in range(5):
    #score = w1+w2+...
    data = news_data["title"][i]
    accum = 0 
    #mood_accum = np.array(10)
    #Negative 	Positive 	Uncertainty 	Litigious 	Constraining 	Superfluous 	Interesting 	Modal 	Irr_Verb 	Harvard_IV
    Negative = 0
    Positive = 0
    Uncertainty = 0
    Litigious = 0 
    Constraining = 0
    Superfluous = 0 
    Interesting = 0 
    Modal = 0 
    Irr_Verb = 0
    Harvard_IV = 0
    
    tokens = nltk.word_tokenize(data) #Tokenize each news header
    tokens = [w.lower() for w in tokens] #Convert to lower case
    
    stop_words = set(stopwords.words('english')) #Remove stop word
    words = [w for w in tokens if not w in stop_words]
    
    words = [w.upper() for w in words] #Convert to upper case since the dictionary is in uppercase 
    words = [a for a in words if a in main_word] #Match only those word in the main_Word
    
    #Calculate score for each news header
    #Get word proportion of each matched token
    for token in words:
        temp_score = LMD.loc[LMD.Word == token,  'Word Proportion'].values
        #list of list to list
        temp_score = temp_score[0]
        
        accum = accum + temp_score
        #Not a good idea to use numpy vstack in the loop so we will use append with list here
        mood = LMD.loc[LMD.Word == token,  ['Negative' , 'Positive' , 'Uncertainty' ,'Litigious' , 'Constraining' , 'Superfluous' , 'Interesting' , 'Modal' , 'Irr_Verb' , 'Harvard_IV'  ]].values[0]
        
        Negative += mood[0]
        Positive += mood[1]
        Uncertainty += mood[2]
        Litigious += mood[3]
        Constraining += mood[4]
        Superfluous += mood[5]
        Interesting += mood[6]
        Modal += mood[7]
        Irr_Verb +=  mood[8]
        Harvard_IV +=  mood[9]
    
    
    final_score.append(accum)
    
    Negative_final.append(Negative)
    Positive_final.append(Positive)
    
    Uncertainty_final.append(Uncertainty)
    Litigious_final.append(Litigious)
    
    Constraining_final.append(Constraining)
    Superfluous_final.append(Superfluous)
    
    Interesting_final.append(Interesting)
    Modal_final.append(Modal)
    
    Irr_Verb_final.append(Irr_Verb)
    Harvard_IV_final.append(Harvard_IV)

In [12]:
#building the feature of the dataset
news_data["score"] = final_score
news_data["Negative_final"] = Negative_final
news_data["Positive_final"] = Positive_final
news_data["Uncertainty_final"] = Uncertainty_final
news_data["Litigious_final"] = Litigious_final
news_data["Constraining_final"] = Constraining_final
news_data["Superfluous_final"] = Superfluous_final
news_data["Interesting_final"] = Interesting_final
news_data["Modal_final"] = Modal_final
news_data["Irr_Verb_final"] = Irr_Verb_final
news_data["Harvard_IV_final"] = Harvard_IV_final
news_data["final_date"] = final_date


In [15]:
news_data_final=news_data.drop(news_data.columns[:3], axis=1)

In [123]:
s2012=pd.read_csv('score.csv')
s2013=pd.read_csv("WSJ2013-2018score/score2013.csv")
s2014=pd.read_csv("WSJ2013-2018score/score2014.csv")
s2015=pd.read_csv("WSJ2013-2018score/score2015.csv")
s2016=pd.read_csv("WSJ2013-2018score/score2016.csv")
s2017=pd.read_csv("WSJ2013-2018score/score2017.csv")

In [124]:
sums.isnull().any()

Constraining_final    False
Harvard_IV_final      False
Interesting_final     False
Irr_Verb_final        False
Litigious_final       False
Modal_final           False
Negative_final        False
Positive_final        False
Superfluous_final     False
Uncertainty_final     False
len                    True
score                 False
dtype: bool

In [125]:
s2017['final_date']=s2017.final_date.str.replace('\r\n','')
s2017['final_date']=s2017.final_date.str.replace('\r','')
s2017['len']=s2017.final_date.apply(len)
s2017=s2017[s2017['len']==11]
del s2017['len']

In [126]:
s2015['len']=s2015.final_date.apply(len)
s2015=s2015[s2015.len != 6]
del s2015['len']

In [127]:
sumdata=pd.concat([s2012,s2013,s2014,s2015,s2016,s2017])
sums=sumdata.iloc[:,[0,1,2,3,4,5,6,7,8,9,-3,-2]]
#final=pd.concat([sums,news_data_final])
sums.to_csv('final.csv')

In [135]:
sums['Interesting_final'].value_counts()

0    150189
1     93741
2     32243
3      6635
4       920
5        95
6        10
Name: Interesting_final, dtype: int64

In [132]:
sums['Constraining_final'] = sums['Constraining_final'].astype(str) 
sums['Constraining_final'] = sums['Constraining_final'].replace('2009','1')
sums['Constraining_final'] = sums['Constraining_final'].replace('2011','1')
sums['Constraining_final'] = sums['Constraining_final'].replace('4018','2')
sums['Constraining_final'] = sums['Constraining_final'].replace('6027','3')



/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [134]:
sums.to_csv('final.csv')